In [ ]:
import csv

from Scripts.scraping import get_data

csv.field_size_limit(5000000)
import ast
import operator
import re


import requests
from tqdm import tqdm
from bs4 import BeautifulSoup

from concurrent.futures import ThreadPoolExecutor
import concurrent

import random
import time




In [ ]:
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.122 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:79.0) Gecko/20100101 Firefox/79.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.1 Safari/605.1.15",
]


# Function to get page content
def get_data(url):
    headers = {"User-Agent": random.choice(USER_AGENTS)}  # Rotate user-agent

    try:
        response = requests.get(url, headers=headers, timeout=3)

        # Handle rate-limiting (HTTP 429) by pausing and retrying
        if response.status_code == 429:
            tqdm.write(f"FROM GET_DATA: Rate limit reached. Sleeping before retrying {url}")
            # print(f"FROM GET_DATA: Rate limit reached. Sleeping before retrying {url}")
            time.sleep(random.uniform(4, 8))  # Random delay to avoid detection
            return get_data(url)

        if response.status_code == 200:
            return response.content  # Return HTML content if successful

        tqdm.write(f"FROM GET_DATA: Failed to retrieve {url}, Status Code: {response.status_code}")
        # print(f"FROM GET_DATA: Failed to retrieve {url}, Status Code: {response.status_code}")
        return None

    except requests.RequestException as e:
        tqdm.write(f"FROM GET_DATA: Error fetching {url}: {e}")
        # print(f"FROM GET_DATA: Error fetching {url}: {e}")
        return None

def read_links_from_csv(csv_filename):
    """Writes the current state of the links dictionary to a CSV file."""
    links = {}
    with open(csv_filename, "r", newline='') as csvfile:
        csvwriter = csv.reader(csvfile)
        for row in csvwriter:
            links[row[0]] = ast.literal_eval(row[1])
    return links


In [ ]:


def has_letters(input_string):
    return any(char.isalpha() for char in input_string)

def soup_mapper(soup):
    word_tag_tuples = []
    # Iterate through all tags
    for tag in soup.descendants:
        # Look only for deepest tags that dont contain other tags
        if tag.name and not tag.find_all():
            # Extract the word and tag
            for word in tag.text.split():  
                word_tag_tuples.append((word, tag))
    
    if len(word_tag_tuples) == 0:
        return None
    return word_tag_tuples

def formated_link_content(link):
    html_data = get_data(link)
    if html_data is None:
        return None
    
    soup = BeautifulSoup(html_data, "html.parser")
    # Finding the title in the meta tags
    title = soup.find('title')
    if title:
        title = title.get_text()
    else: title = None
    
    # Remove scripts, styles, and irrelevant content
    for script in soup(["script", "style", "footer", "nav", "header", "noscript", "head"]):
        script.extract()

    word_tag_tuples = soup_mapper(soup) # by joining word_tag_tuples[0] you get the full text
    
    url_index = link.rfind('/')
    url_last_path = link[url_index + 1:].replace('-', ' ').replace('_', ' ')
    
    if not has_letters(url_last_path):
        url_last_path = None
    
    return word_tag_tuples, title, url_last_path, soup


formated_link_content("https://www.ikea.com/ro/ro/p/styrspel-scaun-de-gaming-purpuriu-negru-20522027/")

## TESTS

In [16]:

html_data = "<html><body><h1 class='something'>Title</h1><p>Food is cool</p><div>Another text</div></body></html>"
soup = BeautifulSoup(html_data, "html.parser")

word_tag_tuples = []

# Iterate through all tags
for tag in soup.descendants:
    # Look only for deepest tags that dont contain other tags
    if tag.name and not tag.find_all():
        # Extract the word and tag
        for word in tag.text.split():  
            word_tag_tuples.append((word, tag))

# Print the result
for word, tag in word_tag_tuples:
    print(f"Word: {word}, Tag: {tag}")
    
    
print(soup.getText(separator=' '))

Word: Title, Tag: <h1 class="something">Title</h1>
Word: Food, Tag: <p>Food is cool</p>
Word: is, Tag: <p>Food is cool</p>
Word: cool, Tag: <p>Food is cool</p>
Word: Another, Tag: <div>Another text</div>
Word: text, Tag: <div>Another text</div>
Title Food is cool Another text
